In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
btvote = pd.read_pickle('../data/btvote.pkl')
btvote.head()

,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,yes
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,yes,yes,yes,yes,yes,yes,no,absence,absence,absence
2,Linke,no,no,no,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,abstain,no,no
3,CDU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,absence
4,Linke,absence,absence,absence,absence,absence,absence,absence,absence,absence,...,no,no,no,no,no,no,no,abstain,no,absence


# Split data and encode target variable

In [8]:
# Split dataframe in 'data' and 'target'
btvote_data = btvote.drop('party', axis=1)
btvote_target = btvote['party']

# Encode the target variable
label_encoder = preprocessing.LabelEncoder()
btvote_target = label_encoder.fit_transform(btvote_target)

In [131]:
label_encoder.classes_

array(['AfD', 'CDU', 'CSU', 'FDP', 'GRÜNE', 'Linke', 'SPD'], dtype=object)

# Pipeline and GridSearch setup

In the pipeline we include the SimpleImputer with the strategy 'most_frequent' as a result of the experiments in the previous notebook.\
As an estimator, we consider:
- k-NearestNeighbor
- NearestCentroid
- Decision Tree
- Naive Bayes

In [13]:
from imblearn.pipeline import Pipeline
# normalisation
from sklearn.preprocessing import OneHotEncoder
# imputer
from sklearn.impute import SimpleImputer
# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# create Pipeline
pipeline = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)), ('estimator', None)])

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# specify the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# define the scoring function
# Note: We use 'macro' as average as we want to evaluate the performance of each class equally, regardless of the class size
f1 = make_scorer(f1_score, average='macro')

# k-NN vs. Nearest Centroid

In [15]:
# set parameter grid
parameters = [
    {
        'estimator': [KNeighborsClassifier()],
        'estimator__n_neighbors': range(2, 10),
        'estimator__weights': ['uniform', 'distance'],
        'estimator__p': [1, 2]  # For Minkowski distance (1 for Manhattan, 2 for Euclidean)'
    }, {
        'estimator': [NearestCentroid()],
        'estimator__metric': ['manhattan', 'euclidean']
    }
]

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklear

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_estimator__n_neighbors,param_estimator__p,param_estimator__weights,param_estimator__metric,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.040901,0.001138,0.039051,0.000473,KNeighborsClassifier(),2,1,uniform,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.669746,0.672986,0.693568,0.659148,0.741757,0.628838,0.700802,0.676794,0.029854,22
1,0.043428,0.003525,0.023601,0.000917,KNeighborsClassifier(),2,1,distance,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.696179,0.668992,0.725647,0.699891,0.694843,0.666064,0.711872,0.696597,0.021862,4
2,0.041652,0.001762,0.013301,0.001847,KNeighborsClassifier(),2,2,uniform,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.677551,0.660556,0.678760,0.664160,0.743878,0.636630,0.695054,0.675808,0.028871,23
3,0.049775,0.007233,0.013801,0.001167,KNeighborsClassifier(),2,2,distance,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.696179,0.668992,0.725647,0.699891,0.694843,0.666064,0.711872,0.696597,0.021862,4
4,0.042873,0.001373,0.039780,0.000829,KNeighborsClassifier(),3,1,uniform,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.692245,0.668226,0.646754,0.691899,0.701681,0.649458,0.681406,0.678736,0.019570,21
5,0.047649,0.006729,0.024453,0.001421,KNeighborsClassifier(),3,1,distance,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.711048,0.682458,0.626863,0.712058,0.711872,0.701936,0.640655,0.692635,0.033384,10
6,0.043601,0.002616,0.013799,0.001832,KNeighborsClassifier(),3,2,uniform,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.716298,0.682216,0.633272,0.708248,0.680290,0.713960,0.665393,0.695739,0.030688,7
7,0.052400,0.005218,0.016500,0.002499,KNeighborsClassifier(),3,2,distance,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.711048,0.682458,0.626863,0.712058,0.711872,0.701936,0.640655,0.692635,0.033384,10
8,0.042866,0.002548,0.039629,0.000994,KNeighborsClassifier(),4,1,uniform,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.661579,0.659071,0.658543,0.671150,0.690461,0.688210,0.671501,0.679636,0.019800,19
9,0.048001,0.006434,0.024300,0.001347,KNeighborsClassifier(),4,1,distance,NaN,"{'estimator': KNeighborsClassifier(), 'estimat...",...,0.709447,0.672381,0.656436,0.690110,0.700886,0.671177,0.702138,0.687910,0.015785,12


From this table it is clearly visible that NearestCentroid works better than NearestNeighbor. Out of 34 combinations, the two combinations with NearestCentroid() are the two best. The mean test score is on average 0.02-0.03 better. We see, that the mean test score for NearestCentroid is better when using euclidean distance rather than manhatten distance. 

It makes sense that NearestCentroid performs better than k-NN as our dataset includes some label noise. In that case k-NN loses performance while Nearest Centroid is stable.

From now on, NearestNeighbor will be neglected. The NearestCentroid approach with distance euclidean will be considered in later experiments.

# DecisionTree analysis

In [117]:
# set parameter grid
parameters = {
    'estimator': [DecisionTreeClassifier()],
    'estimator__criterion': ['gini', 'entropy'],
    'estimator__max_depth': [None, 5, 10, 15],
    'estimator__min_samples_split': [2, 5, 10],
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['criterion'] = results['param_estimator__criterion'].astype(str)
results['max_depth'] = results['param_estimator__max_depth'].astype(str)
results['min_samples_split'] = results['param_estimator__min_samples_split']
pivoted_results = results.pivot(index=['criterion','max_depth'], columns='min_samples_split', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc[('Average',''),:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklear

min_samples_split           2         5        10   Average
criterion max_depth                                        
entropy   10         0.706294  0.706894  0.704596  0.705928
          15         0.695392  0.695852  0.708559  0.699934
          5          0.686869  0.680051  0.684580  0.683833
          None       0.696418  0.700179  0.705764  0.700787
gini      10         0.708531  0.699010  0.690861  0.699467
          15         0.708727  0.706496  0.692139  0.702454
          5          0.672871  0.667768  0.656071  0.665570
          None       0.700743  0.684965  0.698402  0.694703
Average              0.696980  0.692652  0.692622  0.694085

In [113]:
btvote_data
# Manually handle imputation and one-hot encoding
imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = imputer.fit_transform(btvote_data)

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_encoded = encoder.fit_transform(X_train_imputed)
X_train_encoded.shape

C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(745, 911)

Observations:
- max_depth: choosing a max_depth of 5 works rather badly. The results of all other approaches are similar
- min_samples_split: On average, there are no real differences, 10 works slightly less good.
- criterion: the gini index is more prone to smaller max_depth. For larger max_depth the average is similar, the top values can be achieved for the gini index.

We choose criterion='gini', max_depth=15 and min_samples_split=5 as the best parameter setting for a DecisionTreeClassifier.

In [87]:
grid_search_estimator.best_estimator_
joblib.dump(grid_search_estimator.best_estimator_, "decision_tree.joblib")

['decision_tree.joblib']

In [121]:
feature_names = grid_search_estimator.best_estimator_[1].get_feature_names_out()

# Sample feature importance vector (replace this with your actual data)
feature_importance_vector = grid_search_estimator.best_estimator_[2].feature_importances_

# Create a dictionary to store aggregated importance for original features
original_feature_importance = {}

# Iterate over one-hot encoded features and their importance values
for encoded_feature, importance in zip(feature_names, feature_importance_vector):
    # Extract the original feature name
    original_feature = encoded_feature.split('_')[0]

    # Aggregate importance for original features
    if original_feature in original_feature_importance:
        original_feature_importance[original_feature] += importance
    else:
        original_feature_importance[original_feature] = importance

# Normalize importance values
total_importance = sum(original_feature_importance.values())
normalized_feature_importance = {feature: importance / total_importance for feature, importance in original_feature_importance.items()}

print(normalized_feature_importance)


{'x0': 0.0, 'x1': 0.019441511093890293, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 0.018559697146304677, 'x6': 0.0, 'x7': 0.0, 'x8': 0.0, 'x9': 0.0, 'x10': 0.004592060508190742, 'x11': 0.002568873933743104, 'x12': 0.007175130642523832, 'x13': 0.00596891940372288, 'x14': 0.0, 'x15': 0.0, 'x16': 0.0028899831754609926, 'x17': 0.0, 'x18': 0.003853310900614656, 'x19': 0.008909185300621925, 'x20': 0.0, 'x21': 0.0, 'x22': 0.0, 'x23': 0.0034894290609178077, 'x24': 0.0, 'x25': 0.0, 'x26': 0.13351641508220463, 'x27': 0.0, 'x28': 0.0, 'x29': 0.0, 'x30': 0.0, 'x31': 0.0, 'x32': 0.0, 'x33': 0.0, 'x34': 0.002664459940580059, 'x35': 0.00687850284481055, 'x36': 0.0, 'x37': 0.00976498819647021, 'x38': 0.0, 'x39': 0.20207974410467744, 'x40': 0.0, 'x41': 0.00294252832410574, 'x42': 0.0, 'x43': 0.0027401321959926433, 'x44': 0.0, 'x45': 0.0, 'x46': 0.0, 'x47': 0.0, 'x48': 0.0, 'x49': 0.002935855924277834, 'x50': 0.0, 'x51': 0.0, 'x52': 0.0, 'x53': 0.005020980870497884, 'x54': 0.0, 'x55': 0.011815945312566446, 

In [128]:
sorted_dict = dict(sorted(normalized_feature_importance.items(), key=lambda item: item[1], reverse=True))
[x for x in sorted_dict][:10]

['x39', 'x170', 'x26', 'x139', 'x164', 'x1', 'x5', 'x148', 'x231', 'x230']

# NearestCentroid vs. DecisionTree vs. NaiveBayes

In [104]:
# set parameter grid
parameters = {
    'estimator': [NearestCentroid(metric='euclidean'),
                  DecisionTreeClassifier(criterion='gini', max_depth=15, min_samples_split=5),
                  GaussianNB()
                  ]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results[['param_estimator','mean_test_score','std_test_score','rank_test_score']])

C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\david\Documents\Projects\Data Mining\btvote\venv\lib\site-packages\sklear

,param_estimator,mean_test_score,std_test_score,rank_test_score
0,NearestCentroid(),0.717709,0.024862,1
1,"DecisionTreeClassifier(max_depth=15, min_sampl...",0.699968,0.023781,3
2,GaussianNB(),0.705059,0.036818,2


The Grid Search cross validation shows a slight advantage for the NearestCentroid estimator, while the DecisionTreeClassifier and the NaiveBayes approach are equally good.
We will validate the results by performing a *nested* cross validation for each of the appraoches. 

In [11]:
from sklearn.model_selection import cross_val_score

# We use stratified 10 fold cross validation for the inner and the outer fold.
print('Nested Cross Validation Mean scores:')

# NearestCentroid
pipeline.set_params(estimator=NearestCentroid(metric='euclidean'))
print('- NearestCentroid: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# NearestCentroid
pipeline.set_params(estimator=DecisionTreeClassifier(criterion='gini', max_depth=15, min_samples_split=5))
print('- DecisionTree: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# NearestCentroid
pipeline.set_params(estimator=GaussianNB())
print('- NaiveBayes: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

Nested Cross Validation Mean scores:
- NearestCentroid: 0.7177086380839047
- DecisionTree: 0.705414996108032
- NaiveBayes: 0.7050591514452207


We see our results from before validated.

# Summary

Until now, we evaluated:
- k-NearestNeighbor
- NearestCentroid
- Decision Tree
- Naive Bayes

We've seen that out of these four model families, the model NearestCentroid with the metric 'euclidean' performs the best with a F1-score of approx. 0.7177

# Extended model evaluation

We will now compare the NearestCentroid to some more complex approaches, that aren't dealt with in detail in the lecture slides or the exercises.

## Neural network: Multi-Layer Perceptron

In [22]:
from sklearn.neural_network import MLPClassifier

# set parameter grid
parameters = {
    'estimator': [MLPClassifier()],
    'estimator__hidden_layer_sizes': [(50,), (50, 50), (100, 50, 25),], # Specify the architecture of hidden layers
    'estimator__alpha': [0.0001, 0.001, 0.01], # L2 regularization term
    'estimator__random_state': [42],  # Random seed for reproducibility
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['hidden_layer_sizes'] = results['param_estimator__hidden_layer_sizes'].astype(str)
results['alpha'] = results['param_estimator__alpha']
pivoted_results = results.pivot(index='hidden_layer_sizes', columns='alpha', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

c:\Users\mogra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mogra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mogra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mogra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

alpha,0.0001,0.001,0.01,Average
hidden_layer_sizes,,,,
"(100, 50, 25)",0.727441,0.730985,0.727006,0.728477
"(50, 50)",0.722445,0.727712,0.722190,0.724116
"(50,)",0.726010,0.724334,0.721371,0.723905
Average,0.725299,0.727677,0.723522,0.725499


We see that using three (or generally more) hidden layers leeds to a higher F1-Score. We now try some larger and wider hidden layers and increase the maximum iterations, as we've occured some warnings, that the maximum number of iterations was reached without convergence. The alpha parameter doesn't effectively change the results, so we will neglect it in the next experiment and just use the default value. 

In [24]:
# set parameter grid
parameters = {
    'estimator': [MLPClassifier()],
    'estimator__hidden_layer_sizes': [(100, 50, 25),(100, 50, 25, 10),(150, 100, 50, 25, 10)], # Specify the architecture of hidden layers
    'estimator__max_iter': [200,300], # Maximum number of gradient steps.
    'estimator__random_state': [42],  # Random seed for reproducibility
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['hidden_layer_sizes'] = results['param_estimator__hidden_layer_sizes'].astype(str)
results['max_iter'] = results['param_estimator__max_iter']
pivoted_results = results.pivot(index='hidden_layer_sizes', columns='max_iter', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
display(pivoted_results)

max_iter,200,300,Average
hidden_layer_sizes,,,
"(100, 50, 25)",0.727441,0.727441,0.727441
"(100, 50, 25, 10)",0.743029,0.743029,0.743029
"(150, 100, 50, 25, 10)",0.732890,0.732890,0.732890


We see that a MLP with four hidden layers results in the highest F1-score. In that case, increasing the maximum number of iterations is neither needed for convergence nor beneficial for the overall result.

## RandomForestClassifier

At first, we analyse the criterion and how the number of trees affects the result:

In [33]:
from sklearn.ensemble import RandomForestClassifier

# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['gini', 'entropy'],  # Function to measure the quality of a split
    'estimator__n_estimators': [10, 50, 100, 150, 200],  # Number of trees in the forest
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['criterion'] = results['param_estimator__criterion'].astype(str)
results['n_estimators'] = results['param_estimator__n_estimators']
pivoted_results = results.pivot(index='criterion', columns='n_estimators', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_29676\3257803325.py:23: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index='criterion', columns='n_estimators', values='mean_test_score')


n_estimators,10,50,100,150,200,Average
criterion,,,,,,
entropy,0.713631,0.715985,0.720828,0.720625,0.725383,0.719290
gini,0.715285,0.721629,0.713958,0.716208,0.721429,0.717702
Average,0.714458,0.718807,0.717393,0.718417,0.723406,0.718496


The criterion 'entropy' works slightly better than 'gini'. Increasing the n_estimators above 100 (which is the default value) doesn't significantly improve the F1-score. So, we will use the pair ('entropy',100) to evaluate the effect of max_depth and min_samples_split.

In [34]:
# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['entropy'],
    'estimator__max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'estimator__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# transform the results for better visualization
results['max_depth'] = results['param_estimator__max_depth']
results['min_samples_split'] = results['param_estimator__min_samples_split']
pivoted_results = results.pivot(index='max_depth', columns='min_samples_split', values='mean_test_score')
pivoted_results['Average'] = pivoted_results.mean(axis=1)
pivoted_results.loc['Average',:] = pivoted_results.mean()
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_29676\914807656.py:23: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index='max_depth', columns='min_samples_split', values='mean_test_score')


min_samples_split,2,5,10,Average
max_depth,,,,
NaN,0.720828,0.715816,0.723665,0.720103
10.0,0.726481,0.717970,0.719913,0.721455
20.0,0.732263,0.727962,0.728672,0.729632
30.0,0.739289,0.729779,0.730795,0.733287
Average,0.729715,0.722882,0.725761,0.726119


The results don't differ much for different min_samples_split values, so we will keep the default value 2. A larger max_depth seams to increase the F1-score to a certain extent, but if the depth is to large (max_depth=None) the results are worse again. We will further analyse this now.

In [35]:
# set parameter grid
parameters = {
    'estimator': [RandomForestClassifier()],
    'estimator__criterion': ['entropy'],
    'estimator__max_depth': [10, 20, 30, 40, 50, 60, None],  # Maximum depth of the trees
    'estimator__random_state': [42]
}

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results[['param_estimator__max_depth','mean_test_score','std_test_score','rank_test_score']])

,param_estimator__max_depth,mean_test_score,std_test_score,rank_test_score
0,10,0.726481,0.026108,4
1,20,0.732263,0.025497,2
2,30,0.739289,0.019331,1
3,40,0.731884,0.011489,3
4,50,0.723900,0.017138,5
5,60,0.721387,0.014123,6
6,None,0.720828,0.014265,7


The maximum test score is achieved for max_depth=30.

## Summary

We finally perform a nested cross validation for MLP and RandomForestClassifier to compare it to NearestCentroid.

In [36]:
# We use stratified 10 fold cross validation for the inner and the outer fold.
print('Nested Cross Validation Mean scores:')

# NearestCentroid
pipeline.set_params(estimator=NearestCentroid(metric='euclidean'))
print('- NearestCentroid: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# MLPClassifier
pipeline.set_params(estimator=MLPClassifier(hidden_layer_sizes=(100,50,25,10), random_state=42))
print('- MLPClassifier: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

# RandomForestClassifier
pipeline.set_params(estimator=RandomForestClassifier(criterion='entropy', max_depth=30, random_state=42))
print('- RandomForestClassifier: {}'.format(cross_val_score(pipeline, btvote_data, btvote_target, cv=stratified_10_fold_cv, scoring=f1).mean()))

Nested Cross Validation Mean scores:
- NearestCentroid: 0.7177086380839047
- MLPClassifier: 0.7430289957523376
- RandomForestClassifier: 0.7392888391424595


The MLP with hidden layers (100, 50, 25, 10) can achieve the highest F1-Score with 0.74. Close behind we have the RandomForestClassifier and lastly the Nearest Centroid.